<a href="https://colab.research.google.com/github/abhishekm99/sentiment_analysis_am/blob/main/Sentiment_Anlaysis_and_Anamoly_Detection_In_Election_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocess_tweet(tweet):
    # Remove URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)
    
    # Remove special characters and numbers
    tweet = re.sub(r"[^\w\s]", "", tweet)
    tweet = re.sub(r"\d+", "", tweet)
    
    # Convert to lowercase
    tweet = tweet.lower()
    
    # Tokenize the tweet
    tokens = word_tokenize(tweet)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into a single string
    processed_tweet = " ".join(tokens)
    
    return processed_tweet


In [ ]:
import tweepy
from textblob import TextBlob

# Twitter API v2 credentials
bearer_token = "",
consumer_secret = ""
consumer_key = ""
access_token = ""
access_token_secret = ""
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key, consumer_secret=consumer_secret,
    access_token=access_token, access_token_secret=access_token_secret
)

# Collect tweets
topic = "Election"  # Specify the topic you want to analyze
tweet_count = 100  # Number of tweets to collect

tweets = client.search_recent_tweets(query=topic, tweet_fields=["created_at"], max_results=tweet_count, user_auth=True)

# Perform sentiment analysis
positive_tweets = 0
negative_tweets = 0
neutral_tweets = 0

for tweet in tweets.data:
    cleaned_tweet = preprocess_tweet(tweet.text)  # Implement the preprocess_tweet() function to clean the tweet text
    
    # Perform sentiment analysis using TextBlob
    analysis = TextBlob(cleaned_tweet)
    sentiment_score = analysis.sentiment.polarity
    
    if sentiment_score > 0:
        positive_tweets += 1
    elif sentiment_score < 0:
        negative_tweets += 1
    else:
        neutral_tweets += 1

# Calculate sentiment percentages
total_tweets = positive_tweets + negative_tweets + neutral_tweets
positive_percentage = (positive_tweets / total_tweets) * 100
negative_percentage = (negative_tweets / total_tweets) * 100
neutral_percentage = (neutral_tweets / total_tweets) * 100

# Print sentiment distribution
print("Sentiment Distribution:")
print("Positive: {:.2f}%".format(positive_percentage))
print("Negative: {:.2f}%".format(negative_percentage))
print("Neutral: {:.2f}%".format(neutral_percentage))


detect_anomaly(tweets)


In [ ]:
def detect_anomaly(tweets):
    # Perform sentiment analysis and collect scores
  sentiment_scores = []

  for tweet in tweets.data:
      cleaned_tweet = preprocess_tweet(tweet.text)  # Implement the preprocess_tweet() function to clean the tweet text
      
      # Perform sentiment analysis using TextBlob
      analysis = TextBlob(cleaned_tweet)
      sentiment_score = analysis.sentiment.polarity
      sentiment_scores.append(sentiment_score)

  # Convert sentiment scores to numpy array
  sentiment_scores = np.array(sentiment_scores).reshape(-1, 1)

  # Perform anomaly detection using Isolation Forest
  isolation_forest = IsolationForest(contamination=0.1)
  isolation_forest.fit(sentiment_scores)

  # Predict anomalies
  anomaly_predictions = isolation_forest.predict(sentiment_scores)

  # Find the indices of anomalies
  anomaly_indices = np.where(anomaly_predictions == -1)[0]

  # Print the anomalous tweets
  print("Anomalous Tweets:")
  for index in anomaly_indices:
      print(tweets.data[index].text)
      print("Sentiment Score:", sentiment_scores[index][0])
      print("-------------------------")
